In [ ]:
curl 'https://app.qualp.com.br/api/site/router?json=%7B%22login_cod%22%3A%22%22%2C%22waypoints%22%3A%5B%22junidai%22%2C%22recife%22%2C%22jaboatao%22%5D%2C%22config_rota%22%3A%7B%22volta%22%3Afalse%2C%22costing%22%3A%22bus%22%2C%22costing_options%22%3A%7B%22bus%22%3A%7B%22shortest%22%3Afalse%2C%22toll_booth_penalty%22%3A0%2C%22top_speed%22%3A140%7D%7D%2C%22directions_options%22%3A%7B%22units%22%3A%22km%22%2C%22language%22%3A%22pt-BR%22%2C%22directions_type%22%3A%22maneuvers%22%2C%22narrative%22%3Atrue%7D%7D%2C%22config_veiculo%22%3A%7B%22categoria%22%3A%22truck%22%2C%22eixos%22%3A6%7D%2C%22config_pedagio%22%3A%7B%22prices_from_date%22%3A%22%22%7D%2C%22consumo_combustivel%22%3A%7B%22preco%22%3A0%2C%22consumo%22%3A0%7D%2C%22mc%22%3A%22mc%22%2C%22type_route%22%3A%22default%22%7D' \
  -H 'Accept: application/json' \
  -H 'Accept-Language: pt-BR,pt;q=0.7' \
  -H 'Access-Control-Allow-Origin: *' \
  -H 'Connection: keep-alive' \
  -H 'Origin: https://qualp.com.br' \
  -H 'Referer: https://qualp.com.br/' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'Sec-GPC: 1' \
  -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36' \
  -H 'sec-ch-ua: "Chromium";v="122", "Not(A:Brand";v="24", "Brave";v="122"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"'

In [ ]:
import requests

def calcular_rota(*waypoints):
    # Verifica se pelo menos dois waypoints foram fornecidos
    if len(waypoints) < 2:
        print("É necessário fornecer pelo menos dois pontos de passagem.")
        return

    # Constrói o payload JSON com os waypoints fornecidos
    payload = {
        "login_cod": "",
        "waypoints": list(waypoints),  # Converte a tupla de waypoints em uma lista
        # As demais chaves do payload podem ser deixadas vazias ou com valores padrão,
        # dependendo dos requisitos da API
    }

    # Faz a solicitação para o endpoint da API
    response = requests.get('https://app.qualp.com.br/api/site/router', params={'json': payload})

    # Processa a resposta
    if response.status_code == 200:
        print("Rota calculada com sucesso!")
        print("Resposta da API:", response.json())
    else:
        print("Falha ao calcular rota. Código de status:", response.status_code)

# Exemplo de uso da função
calcular_rota("junidai", "recife", "jaboatao")
